In [19]:
import sys
import json
import time

In [2]:
characters = json.load(open("characters.json", "r"))
keys = list(characters.keys())

for c in keys:
    characters[int(c)] = characters[c]
    del characters[c]

In [13]:
target_characters = []
character_ids = []

for anime_id in characters.keys():
    for character in characters[anime_id]:
        if character["role"] == "MAIN":
            if "native" in character["name"]:
                name = character["name"]["native"]
            else:
                name = character["name"]["full"]

            if character["id"] in character_ids:
                continue

            target_characters.append({
                "anime_id": anime_id,
                "character_id": character["id"],
                "name": name,
            })
            character_ids.append(character["id"])

In [17]:
ignorelist = []

In [18]:
target_characters[0]

{'anime_id': 150672, 'character_id': 188783, 'name': '有馬かな'}

In [40]:
from anilist import Client, types, utils
from anilist.types import Character
from anilist.utils import CHARACTER_GET_QUERY, API_URL, HEADERS
import httpx

class CustomClient(Client):
    def get_character(self, id: int) -> types.Character | None:
        if not self.httpx:
            self.httpx = httpx.Client()
        response = self.httpx.post(
            url=API_URL,
            json=dict(
                query=CHARACTER_GET_QUERY,
                variables=dict(
                    id=id,
                ),
            ),
            headers=HEADERS,
        )
        # if response.status_code != 200:
        print(response.headers)
        data = response.json()
        if data["data"]:
            try:
                item = data["data"]["Character"]
                return Character(
                    id=item["id"],
                    name=item["name"],
                    image=item["image"],
                    url=item["siteUrl"],
                    favorites=item["favourites"],
                    description=item["description"],
                    media=item["media"],
                    is_favorite=item["isFavourite"],
                )
            except Exception:
                pass
        return None

client = CustomClient()

for i, chara in enumerate(target_characters):
    sys.stdout.write(f"\r[ {i} / {len(target_characters)} ]")

    if "image_url_medium" in target_characters[i]:
        continue

    res = client.get_character(chara["character_id"])

    if res is None:
        display("broke")
        break

    res = res.raw()

    target_characters[i]["image_url_medium"] = res["image"].raw()["medium"]
    target_characters[i]["image_url_large"] = res["image"].raw()["large"]

    time.sleep(2)


[ 4602 / 5668 ]Headers({'date': 'Thu, 16 May 2024 23:10:19 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'cache-control': 'no-cache, private', 'x-ratelimit-limit': '90', 'x-ratelimit-remaining': '83', 'set-cookie': 'laravel_session=ifwKM3ztD8nf9h9dFpfUbKxQRxrYUYnhkFW1N3uv; expires=Fri, 17-May-2024 11:10:18 GMT; Max-Age=43200; path=/; httponly', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'x-content-type-options': 'nosniff', 'access-control-allow-origin': '*', 'access-control-allow-methods': 'GET, POST, OPTIONS', 'access-control-allow-headers': 'Authorization,Accept,DNT,X-CustomHeader,Keep-Alive,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Content-Range,Range', 'access-control-expose-headers': 'X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, Content-Length, Content-Range', 'cf-cache-status': 'DYNAMIC', 'report-to': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare

In [42]:
json.dump(target_characters, open("characters-with-url.json", "w"))